In [1]:
import aocd
import re
import numpy as np
from collections import deque

In [2]:
class Scanner():
    def __init__(self, id, beacons, position=[0, 0, 0]):
        self.id = id
        self.position = np.array(position)
        self._beacons = np.array(beacons)
        self._rotation = 0

        vectors = []
        for i, b in enumerate(self._beacons):
            vectors.extend(self._beacons[i+1:]-b)
        self._vectors = np.array(vectors)
        self._distances = np.linalg.norm(vectors, axis=1)
    
    @property
    def beacons(self):
        return self._beacons + self.position
    
    @property
    def beacon_tuples(self):
        return [tuple(b) for b in self.beacons]
    
    def rotate(self):
        # c c c cu c c c cr c c c cu ...
        self._clockwise()
        if self._rotation % 8 == 3:
            self._up()
        elif self._rotation % 8 == 7:
            self._right()
        self._rotation += 1
    
    def _up(self):
        # x y z -> x z -y
        self._beacons = self._beacons[:, [0, 2, 1]] * [1, 1, -1]
        self._vectors = self._vectors[:, [0, 2, 1]] * [1, 1, -1]
    
    def _right(self):
        # x y z -> y -x z
        self._beacons = self._beacons[:, [1, 0, 2]] * [1, -1, 1]
        self._vectors = self._vectors[:, [1, 0, 2]] * [1, -1, 1]

    def _clockwise(self):
        # x y z -> z y -x
        self._beacons = self._beacons[:, [2, 1, 0]] * [1, 1, -1]
        self._vectors = self._vectors[:, [2, 1, 0]] * [1, 1, -1]
    
    def intersect_distances(self, other):
        return np.intersect1d(self._distances, other._distances)
    
    def intersect_vectors(self, other):
        v0 = set(tuple(v) for v in self._vectors)
        v1 = set(tuple(v) for v in other._vectors) | set(tuple(-v) for v in other._vectors)
        return v0 & v1

In [3]:
data = aocd.get_data(day=19)
blocks = (block.splitlines() for block in data.split('\n\n'))
scanners = []

for block in blocks:
    scanner_id = int(re.search(r'\d+', block[0])[0])
    lines = (re.match(r'(-?\d+),(-?\d+),(-?\d+)', line) for line in block[1:])
    beacons = [(int(m[1]), int(m[2]), int(m[3])) for m in lines]
    scanners.append(Scanner(scanner_id, beacons))

In [4]:
scanner0 = scanners[0]
scanners_to_check  = deque([scanner0])
canonical_scanners = set([scanner0])
canonical_beacons  = set(scanner0.beacon_tuples)

while scanners_to_check:
    current_scanner = scanners_to_check.popleft()
    other_scanners = [s for s in scanners if s not in canonical_scanners]

    for other_scanner in other_scanners:
        if len(current_scanner.intersect_distances(other_scanner)) >= 66:
            for _ in range(24):
                other_scanner.rotate()
                if len(current_scanner.intersect_vectors(other_scanner)) >= 66:
                    break
            
            differences = []
            for i, b in enumerate(other_scanner.beacons):
                differences.extend(current_scanner.beacons - b)
            difference, counts = np.unique(differences, return_counts=True, axis=0)
            
            other_scanner.position = difference[np.argmax(counts)]
            scanners_to_check.append(other_scanner)
            canonical_scanners.add(other_scanner)
            canonical_beacons.update(other_scanner.beacon_tuples)

print(len(canonical_beacons))

428


In [5]:
manhattan_distances = []
for i, s1 in enumerate(scanners):
    for s2 in scanners[i+1:]:
        manhattan_distances.append(np.sum(np.abs(s1.position - s2.position)))

print(max(manhattan_distances))

12140


In [6]:
vectors = []
for i, b in enumerate(scanner0._beacons):
    vectors.extend(scanner0._beacons[i+1:]-b)
np.array(vectors).shape

(325, 3)

In [30]:
np.unique(scanner0.beacons[:, None, :] - scanner1.beacons[None, :, :], axis=2, return_counts=True)

(array([[[-2170, -1418, -1008],
         [-1060,  -142,   -76],
         [ -889,  -170, -1160],
         ...,
         [-1839,    34, -1023],
         [-2109, -1452,  -169],
         [-1087, -1235,   199]],
 
        [[-2402,  -141, -2356],
         [-1292,  1135, -1424],
         [-1121,  1107, -2508],
         ...,
         [-2071,  1311, -2371],
         [-2341,  -175, -1517],
         [-1319,    42, -1149]],
 
        [[-1109,     0, -1186],
         [    1,  1276,  -254],
         [  172,  1248, -1338],
         ...,
         [ -778,  1452, -1201],
         [-1048,   -34,  -347],
         [  -26,   183,    21]],
 
        ...,
 
        [[-1410, -1055, -2011],
         [ -300,   221, -1079],
         [ -129,   193, -2163],
         ...,
         [-1079,   397, -2026],
         [-1349, -1089, -1172],
         [ -327,  -872,  -804]],
 
        [[-2588, -1513, -2034],
         [-1478,  -237, -1102],
         [-1307,  -265, -2186],
         ...,
         [-2257,   -61, -2049],
       